In [1]:
from qiskit import QuantumRegister, QuantumCircuit, ClassicalRegister
from qiskit.quantum_info import Statevector, state_fidelity
from qiskit_ibm_runtime.fake_provider import FakeHanoiV2
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
import pickle
import numpy as np


In [2]:
from utils import *
from teleport import *

In [ ]:
#No. of Qubits 
n = 1

#No. of circuits to run
m = 2

In [ ]:
#Prepare list of Initial State
psi_nq = []
for i in range(m):
    psi_nq.append(state_prep(n))

In [ ]:
#Initialize lists

circa = []
cbita = []

circb = []
cbitb = []

mltpl_sva = []
mltpl_svb = []

In [ ]:
#Run both teleportation and checker circuits

for i in range(m):
    qca, ca = teleportation_circuit(n, psi_nq[i])

    mltpl_sva.append(Statevector.from_instruction(qca))
    circa.append(qca)
    cbita.append(ca)

    
    qcb, cb = checker_circuit(n, psi_nq[i])

    mltpl_svb.append(Statevector.from_instruction(qcb))
    circb.append(qcb)
    cbitb.append(cb)
 

In [ ]:
#Measure teach one of them

measure_all(circa, cbita, n)
measure_all(circb, cbitb, n)

In [ ]:
#Load the backend

from qiskit_ibm_runtime import QiskitRuntimeService, Batch

service = QiskitRuntimeService()
backend = service.least_busy(simulator=False, operational=True)

qiskit_runtime_service._resolve_cloud_instances:WARNING:2025-07-24 15:16:06,080: Default instance not set. Searching all available instances.


In [ ]:
#Set number of circuit in a single batch

max_circuit_in_batch = 2

In [ ]:
#Job list for Teleportation circuit

jobsa = simulate_batch(circa, backend, max_circuit_in_batch)

In [ ]:
#Job list for Checker circuit

jobsb = simulate_batch(circb, backend, max_circuit_in_batch)

In [ ]:
#List of counts for Teleportation circuit

mltpl_countsa = get_batch_result(jobsa, n)

In [ ]:
#List of counts for Checker circuit

mltpl_countsb = get_batch_result(jobsb, n)

In [ ]:
#Check fidelity of the counts obtained by the Quantum computer

batch_fdlty = calc_batch_fdlty(mltpl_countsa, mltpl_countsb)

#Check Fidelity of state obtained post teleportation and one before it
batch_state_fdlty = calc_batch_statefdlty(mltpl_sva, mltpl_svb)

In [ ]:
# Save variables to a file

with open('Variables.pkl', 'wb') as f:
    pickle.dump({'mltpl_countsa': mltpl_countsa, 'mltpl_countsb': mltpl_countsb, 'batch_fdlty': batch_fdlty, 'batch_state_fdlty' : batch_state_fdlty, 'n_batch' : n}, f)